In [1]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.insert(0, ".")

In [2]:
df = pd.read_csv('data/train_data_for_students.tsv', sep='\t', header=None)
df.columns = ['id', 'handle', 'tweet', 'date', 'device']
print(f'{df.shape=}')
df.head()

df.shape=(2682, 5)


,id,handle,tweet,date,device
0,845974102619906048,realDonaldTrump,Democrats are smiling in D.C. that the Freedom...,2017-03-26 15:21:58,iphone
1,846166053663191040,realDonaldTrump,General Kelly is doing a great job at the bord...,2017-03-27 04:04:42,iphone
2,835814988686233601,realDonaldTrump,"The race for DNC Chairman was, of course, tota...",2017-02-26 13:33:16,android
3,835817351178301440,realDonaldTrump,For first time the failing @nytimes will take ...,2017-02-26 13:42:39,android
4,835916511944523777,realDonaldTrump,"Russia talk is FAKE NEWS put out by the Dems, ...",2017-02-26 20:16:41,android


In [3]:
df.device.value_counts()


android                                                                                1683
iphone                                                                                  755
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                      201
<a href="http://www.twitter.com" rel="nofollow">Twitter for BlackBerry</a>               13
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>       9
<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>         4
<a href="http://instagram.com" rel="nofollow">Instagram</a>                               3
<a href="https://periscope.tv" rel="nofollow">Periscope.TV</a>                            2
<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>                     1
Name: device, dtype: int64

There is issue with the label column. There are values that not iphone / android. WTF?

In [4]:
df = df[(df.device == 'iphone') | (df.device == 'android')]
df.device.value_counts()

android    1683
iphone      755
Name: device, dtype: int64

In [5]:
# add numric label column
# android = 1
# iphone = 0

df['label'] = 0
df.loc[df['device'] == 'android', 'label'] = 1

In [6]:
from data_processing import create_folds

# Using StratifiedKfold since the label is not that balanced

df = create_folds(df, label_name='device', num_folds=3, seed=11)

input df shape : (2438, 6)
Number of folds: 3, total samples (after removing NaN): 2438
fold: 0, num samples: 813
fold: 1, num samples: 813
fold: 2, num samples: 812


In [7]:
from data_processing import preprocess

# remove urls from tweets snice all hte urls with tweeter shortener

df.tweet = df.tweet.apply(preprocess)

In [34]:
from data_processing import create_datasetdict

dataset = create_datasetdict(df)


Using custom data configuration default-21fd37ae2f0e743e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\itama\.cache\huggingface\datasets\csv\default-21fd37ae2f0e743e\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
from model import tokenize_function

tokenized_datasets = dataset.map(tokenize_function, tokenizer)

  0%|          | 0/2438 [00:00<?, ?ex/s]

TypeError: 'int' object is not callable

In [63]:
sample_id = 2
e = tokenizer.encode_plus(df.loc[sample_id, 'tweet'], padding="max_length", truncation=True)
e['labels'] = [df.loc[sample_id, 'label']]
e.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1625
    })
    test: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 813
    })
})

In [12]:
tokenized_datasets = tokenized_datasets.remove_columns(["tweet"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [13]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8)
eval_dataloader = DataLoader(tokenized_datasets['test'], batch_size=8)

In [14]:
tokenized_datasets['train']

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1625
})

In [15]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [95]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [16]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [17]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [18]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [19]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/612 [00:00<?, ?it/s]

In [43]:
from datasets import load_metric
model.eval()
acc = 0
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    y_pred = torch.argmax(outputs.logits, dim=-1)  # using .logits to change it to Tensor
    y = batch['labels']
    acc += sum(y_pred == y).item()

acc = acc / len(tokenized_datasets['test'])



In [44]:
acc

0.915129151291513

In [69]:
for batch in eval_dataloader:
    
    batch = {k: v.to(device) for k, v in batch.items()}
    for k, v in batch.items():
        print(k, v.shape, type(v))
    break

labels torch.Size([8]) <class 'torch.Tensor'>
input_ids torch.Size([8, 512]) <class 'torch.Tensor'>
attention_mask torch.Size([8, 512]) <class 'torch.Tensor'>


In [88]:
sample_id = 11
e = tokenizer.encode_plus(df.loc[sample_id, 'tweet'], padding="max_length", truncation=True)
e['labels'] = [df.loc[sample_id, 'label']]
df.iloc[sample_id][['tweet', 'label']]

tweet    I am now going to the brand new Trump Internat...
label                                                    1
Name: 11, dtype: object

In [91]:
# batch = {k: torch.tensor(v, dtype=torch.int32, device=device) for k, v in e.items()}
batch = {k: torch.tensor(v).to(device) for k, v in e.items()}

# batch['labels'] = batch['labels'].type(torch.LongTensor).to(device)
batch['attention_mask'] = batch['attention_mask'].unsqueeze(0)
batch['input_ids'] = batch['input_ids'].unsqueeze(0)

for k, v in batch.items():
    print(k, v.shape, type(v))
with torch.no_grad():
    outputs = model(**batch)    
y_pred = torch.argmax(outputs.logits, dim=-1)  # using .logits to change it to Tensor

input_ids torch.Size([1, 512]) <class 'torch.Tensor'>
attention_mask torch.Size([1, 512]) <class 'torch.Tensor'>
labels torch.Size([1]) <class 'torch.Tensor'>


In [87]:
y_pred.item()

1

In [98]:
import transformers
def predict_one(sample_id: int, model: transformers.models) -> int:
    """
    given sample_id from the original df and trained model, the fuction predict the label are return it.
    """
    # tokenizer the sample    sample = tokenizer.encode_plus(df.loc[sample_id, 'tweet'], padding="max_length", truncation=True)
    sample['labels'] = [df.loc[sample_id, 'label']]

    # convert all the sample data to tensors
    sample = {k: torch.tensor(v).to(device) for k, v in e.items()}
    sample['attention_mask'] = sample['attention_mask'].unsqueeze(0)
    sample['input_ids'] = sample['input_ids'].unsqueeze(0)

    model.eval()
    with torch.no_grad():
        outputs = model(**sample)

    y_pred = torch.argmax(outputs.logits, dim=-1)  # using .logits to change it to Tensor
    return y_pred.item()

predict_one(11, model)

1

In [ ]:
sdf